## Liberias y limpieza

### Importar Librerías

In [1]:

# trabajar con dataframes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.spatial.distance import pdist, squareform, cosine

# Librerías auxiliares
# Import linear_kernel



### Limpieza de datos

In [2]:
#df_ini = pd.read_csv("DF2022Ene_Jun_3.csv",delimiter=';')
df_ini = pd.read_csv("Datos/DF2022Ene_Mar_Creditos_Completo.csv",delimiter=';')

In [3]:
# Mostrar los primeros 6 datos
df_ini.head(6)

,creditocodigo,AlmacenCodigo,Categoria,AntiguedadAlmacen,municipioCodigo,Municipio_Almacen,Departamento_Almacen,CodDANE_DptoAlmacen,Canal,PersonaCodigo,...,TasaMora,ValorFactura,UltimaCuotaPagada,FechaUltimoPago,plazo,frecuencia,DescripcionEstadoCredito,valorCuota,CantidadCuotas,FechaCreacionCredito
0,21593606,13394,Zapatos,8,132,Cali,Valle Del Cauca,76,Canal fisico,644573,...,0.019,99953,0,2022-03-10 07:26:59.390,90,30,Pagado,34592,3,2022-01-29 16:07:04.880
1,21592174,15280,Bolsos y Vestuario,7,603,Soacha,Cundinamarca,25,Canal fisico,1324687,...,0.019,74900,0,2022-04-01 15:19:10.133,60,30,Pagado,38521,2,2022-01-29 15:32:57.320
2,21592567,38261,Vestuario,2,130,Bogotá,Bogotá D.c.,11,Ecommerce,4283228,...,0.019,80940,0,2022-05-29 10:50:44.093,120,30,Pagado,21205,4,2022-01-29 15:42:20.190
3,21592967,14043,Deportes,7,220,Florencia,Caqueta,18,Canal fisico,2020091,...,0.019,179000,0,2022-02-04 11:41:20.943,120,30,Pagado,46896,4,2022-01-29 15:52:08.840
4,21592954,12790,Zapatos,8,139,Guadalajara de Buga,Valle Del Cauca,76,Canal fisico,541558,...,0.019,65000,0,2022-04-26 10:38:39.643,90,30,Pagado,22495,3,2022-01-29 15:51:46.927
5,21592652,15525,Vestuario,7,141,Cucuta,Norte De Santander,54,Canal fisico,3874739,...,0.019,39980,0,2022-02-12 15:27:49.630,30,30,Pagado,40740,1,2022-01-29 15:44:14.237


In [4]:
df_ini_credito = pd.read_csv("Datos/DF2022Ene_Mar_Creditos.csv",delimiter=';')
df_ini_almacen = pd.read_csv("Datos/DF2022Ene_Mar_Almacenes.csv",delimiter=';')
df_ini_persona = pd.read_csv("Datos/DF2022Ene_Mar_Personas.csv",delimiter=';')

In [5]:
df_ini_credito.head(6)

,creditocodigo,AlmacenCodigo,PersonaCodigo,DiasMora,SaldoMora,CuotasVencidas,ValorCapital,TasaDescuento,TasaFinanciacion,TasaMora,ValorFactura,UltimaCuotaPagada,FechaUltimoPago,plazo,frecuencia,DescripcionEstadoCredito,valorCuota,CantidadCuotas,FechaCreacionCredito
0,21593606,13394,644573,0,0,0,115670,0.020,0.019,0.019,99953,0,2022-03-10 07:26:59.390,90,30,Pagado,34592,3,2022-01-29 16:07:04.880
1,21592174,15280,1324687,0,0,0,85955,0.010,0.019,0.019,74900,0,2022-04-01 15:19:10.133,60,30,Pagado,38521,2,2022-01-29 15:32:57.320
2,21592567,38261,4283228,0,0,0,94452,0.010,0.019,0.019,80940,0,2022-05-29 10:50:44.093,120,30,Pagado,21205,4,2022-01-29 15:42:20.190
3,21592967,14043,2020091,0,0,0,208885,0.015,0.019,0.019,179000,0,2022-02-04 11:41:20.943,120,30,Pagado,46896,4,2022-01-29 15:52:08.840
4,21592954,12790,541558,0,0,0,75220,0.020,0.019,0.019,65000,0,2022-04-26 10:38:39.643,90,30,Pagado,22495,3,2022-01-29 15:51:46.927
5,21592652,15525,3874739,0,0,0,45498,0.010,0.019,0.019,39980,0,2022-02-12 15:27:49.630,30,30,Pagado,40740,1,2022-01-29 15:44:14.237


In [6]:
df_ini_almacen.head(6)

,AlmacenCodigo,NombreAlmacen,Canal,FechaCreacion,RazonSocial,NombreComercial,Categoria,municipioCodigo,Municipio_Almacen,Departamento_Almacen,CodDANE_DptoAlmacen,AntiguedadAlmacen
0,21083,El Templo De La Moda 2 Cali,Fisico,31/07/2018,El Templo De La Moda S.A.S.,El Templo De La Moda,Vestuario,132,Cali,Valle Del Cauca,76,5
1,16357,Bata BG Florencia 2,Fisico,6/04/2017,CIA Manufacturera Manisol S.A.,Bata,Zapatos,220,Florencia,Caqueta,18,6
2,24327,Surtitodo Mayorca,Fisico,13/03/2019,Compania Comercial Universal S.A.S.,Surtitodo,Vestuario,3,Sabaneta,Antioquia,5,4
3,16346,Totto Santander de Quilichao,Fisico,6/04/2017,PX Retail S.A.S.,Totto,Bolsos y Vestuario,460,Santander De Quilichao,Cauca,19,6
4,1556,Bata Medellin Centro No 5,Fisico,25/10/2012,CIA Manufacturera Manisol S.A.,Bata,Zapatos,1,Medellin,Antioquia,5,11
5,14610,Totto Pamplona,Fisico,23/08/2016,Kalos Comercializadora S.A.S.,Totto,Bolsos y Vestuario,858,Pamplona,Norte De Santander,54,7


In [7]:
df_ini_persona.head(6)

,PersonaCodigo,Edad,Genero
0,644573,47.0,Femenino
1,1324687,53.0,Femenino
2,4283228,36.0,Femenino
3,2020091,60.0,Masculino
4,541558,50.0,Femenino
5,3874739,30.0,Femenino


#### Almacen

In [8]:
#Combinar Municipio y Departamento del almacen
df_ini_almacen['Ubicacion_Almacen'] = df_ini_almacen['Departamento_Almacen'].str.cat(df_ini_almacen['Municipio_Almacen'], sep='-')

In [9]:
df_ini_almacen = df_ini_almacen.astype({'AlmacenCodigo':'object','municipioCodigo':'object','CodDANE_DptoAlmacen':'object'})
df_ini_almacen.dtypes

AlmacenCodigo           object
NombreAlmacen           object
Canal                   object
FechaCreacion           object
RazonSocial             object
NombreComercial         object
Categoria               object
municipioCodigo         object
Municipio_Almacen       object
Departamento_Almacen    object
CodDANE_DptoAlmacen     object
AntiguedadAlmacen        int64
Ubicacion_Almacen       object
dtype: object

In [10]:
print('Cantidad de Filas y columnas de Almacenes:',df_ini_almacen.shape)
print('Nombre columnas de Almacenes:',df_ini_almacen.columns)

Cantidad de Filas y columnas de Almacenes: (1718, 13)
Nombre columnas de Almacenes: Index(['AlmacenCodigo', 'NombreAlmacen', 'Canal', 'FechaCreacion',
       'RazonSocial', 'NombreComercial', 'Categoria', 'municipioCodigo',
       'Municipio_Almacen', 'Departamento_Almacen', 'CodDANE_DptoAlmacen',
       'AntiguedadAlmacen', 'Ubicacion_Almacen'],
      dtype='object')


#### Persona

In [11]:
#Reemplazar los valores de Genero de texto, por numero
df_ini_persona['Genero'] = df_ini_persona['Genero'].replace('F', 1, regex=True)
df_ini_persona['Genero'] = df_ini_persona['Genero'].replace('M', 0, regex=True)
df_ini_persona['Genero'] = df_ini_persona['Genero'].replace(np.NaN, 2, regex=True)

In [12]:
df_ini_persona = df_ini_persona.astype({'PersonaCodigo':'object','Genero':'uint8'})
df_ini_persona.dtypes

PersonaCodigo     object
Edad             float64
Genero             uint8
dtype: object

In [13]:
print('Cantidad de Filas y columnas de Personas:',df_ini_persona.shape)
print('Nombre columnas  de Personas:',df_ini_persona.columns)

Cantidad de Filas y columnas de Personas: (9878, 3)
Nombre columnas  de Personas: Index(['PersonaCodigo', 'Edad', 'Genero'], dtype='object')


#### Creditos

In [14]:
#Reemplazar las comas por puntos para los campos numericos
df_ini_credito['SaldoMora'] = df_ini_credito['SaldoMora'].replace(',', '.', regex=True)
df_ini_credito['ValorCapital'] = df_ini_credito['ValorCapital'].replace(',', '.', regex=True)
df_ini_credito['ValorFactura'] = df_ini_credito['ValorFactura'].replace(',', '.', regex=True)
df_ini_credito['valorCuota'] = df_ini_credito['valorCuota'].replace(',', '.', regex=True)

In [15]:
df_ini_credito = df_ini_credito.astype({'SaldoMora': 'float64', 'ValorCapital': 'float64', 'ValorFactura':'float64', 'valorCuota':'float64', 'creditocodigo':'object', 'AlmacenCodigo':'object','PersonaCodigo':'object','plazo':'float32','frecuencia':'float32','CantidadCuotas':'float32','TasaDescuento':'float32','TasaFinanciacion':'float32','TasaMora':'float32'})
df_ini_credito.dtypes

creditocodigo                object
AlmacenCodigo                object
PersonaCodigo                object
DiasMora                      int64
SaldoMora                   float64
CuotasVencidas                int64
ValorCapital                float64
TasaDescuento               float32
TasaFinanciacion            float32
TasaMora                    float32
ValorFactura                float64
UltimaCuotaPagada             int64
FechaUltimoPago              object
plazo                       float32
frecuencia                  float32
DescripcionEstadoCredito     object
valorCuota                  float64
CantidadCuotas              float32
FechaCreacionCredito         object
dtype: object

In [16]:
print('Cantidad de Filas y columnas:',df_ini_credito.shape)
print('Nombre columnas:',df_ini_credito.columns)

Cantidad de Filas y columnas: (56237, 19)
Nombre columnas: Index(['creditocodigo', 'AlmacenCodigo', 'PersonaCodigo', 'DiasMora',
       'SaldoMora', 'CuotasVencidas', 'ValorCapital', 'TasaDescuento',
       'TasaFinanciacion', 'TasaMora', 'ValorFactura', 'UltimaCuotaPagada',
       'FechaUltimoPago', 'plazo', 'frecuencia', 'DescripcionEstadoCredito',
       'valorCuota', 'CantidadCuotas', 'FechaCreacionCredito'],
      dtype='object')


In [17]:
df_ini_credito.describe()

,DiasMora,SaldoMora,CuotasVencidas,ValorCapital,TasaDescuento,TasaFinanciacion,TasaMora,ValorFactura,UltimaCuotaPagada,plazo,frecuencia,valorCuota,CantidadCuotas
count,56237.000000,5.623700e+04,56237.000000,5.623700e+04,56237.000000,56237.000000,56237.000000,5.623700e+04,56237.000000,56237.000000,56237.000000,5.623700e+04,56237.000000
mean,16.816046,8.477532e+03,0.155752,1.808659e+05,0.012064,0.019342,0.019342,1.547962e+05,0.011309,93.456802,29.695663,5.505319e+04,3.169337
std,78.003615,5.753093e+04,0.820885,1.883753e+05,0.007032,0.002214,0.002214,1.575696e+05,0.249646,48.262875,2.114832,5.262835e+04,1.681946
min,0.000000,0.000000e+00,0.000000,1.133000e+04,0.000000,0.000000,0.000000,1.000000e+04,0.000000,30.000000,15.000000,5.070000e+03,1.000000
25%,0.000000,0.000000e+00,0.000000,7.961600e+04,0.010000,0.019000,0.019000,6.935000e+04,0.000000,60.000000,30.000000,2.879200e+04,2.000000
50%,0.000000,0.000000e+00,0.000000,1.281300e+05,0.010000,0.020000,0.020000,1.099010e+05,0.000000,90.000000,30.000000,4.376000e+04,3.000000
75%,0.000000,0.000000e+00,0.000000,2.112190e+05,0.020000,0.020000,0.020000,1.825000e+05,0.000000,120.000000,30.000000,6.169100e+04,4.000000
max,454.000000,2.396455e+06,16.000000,3.679649e+06,0.100000,0.020000,0.020000,2.935000e+06,16.000000,360.000000,30.000000,1.863948e+06,16.000000


##### Detalle Almacen

In [18]:
df_combined_persona_credito = pd.merge(df_ini_credito[['creditocodigo', 'AlmacenCodigo', 'PersonaCodigo','ValorFactura']], df_ini_persona[['PersonaCodigo','Genero','Edad']], on='PersonaCodigo')
df_combined_persona_credito.head()

,creditocodigo,AlmacenCodigo,PersonaCodigo,ValorFactura,Genero,Edad
0,21593606,13394,644573,99953.0,1,47.0
1,21593103,16197,644573,84900.0,1,47.0
2,21592240,1937,644573,111800.0,1,47.0
3,21815693,40877,644573,250000.0,1,47.0
4,21592174,15280,1324687,74900.0,1,53.0


In [19]:
df_almacenes_variables = df_combined_persona_credito.groupby('AlmacenCodigo').agg(TotalCreditos=('creditocodigo', 'count'),CreditosMujeres=('Genero', 'sum'),cant_clientes=('PersonaCodigo', 'nunique'), edad_clientes=('Edad', 'mean'), Prom_Valorfactura=('ValorFactura', 'mean'))

In [20]:
#2.segundo paso: Calcular las proporciones y eliminar las otras del dataframe de almacenes
df_almacenes_variables['porcentaje_mujeres'] = (df_almacenes_variables['CreditosMujeres'] / df_almacenes_variables['TotalCreditos']) #PROPORCION MUJERES
df_almacenes_variables = df_almacenes_variables.drop('CreditosMujeres', axis=1)
df_almacenes_variables.head()

,TotalCreditos,cant_clientes,edad_clientes,Prom_Valorfactura,porcentaje_mujeres
AlmacenCodigo,,,,,
10,1,1,34.000000,139853.000000,1.0
17,19,8,33.263158,135604.473684,1.0
35,12,9,40.500000,238183.333333,1.0
194,5,2,34.000000,336214.400000,0.8
303,7,4,29.428571,158696.142857,1.0


### Asignación de dataset a df

In [21]:
df_credito = df_ini_credito[['creditocodigo', 'AlmacenCodigo', 'PersonaCodigo']]
df_almacen = df_ini_almacen[['AlmacenCodigo','Categoria','Ubicacion_Almacen','Canal']]
df_persona = df_ini_persona
df_almacen_detalle = pd.merge(df_almacen, df_almacenes_variables, on='AlmacenCodigo')
df_almacen_detalle.head()
#df = df[df['FechaCreacionCredito'].str.startswith('2022-05')]

,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal,TotalCreditos,cant_clientes,edad_clientes,Prom_Valorfactura,porcentaje_mujeres
0,21083,Vestuario,Valle Del Cauca-Cali,Fisico,87,42,33.747126,103334.413793,0.965517
1,16357,Zapatos,Caqueta-Florencia,Fisico,47,39,34.319149,91275.340426,0.851064
2,24327,Vestuario,Antioquia-Sabaneta,Fisico,21,15,35.380952,140498.714286,0.904762
3,16346,Bolsos y Vestuario,Cauca-Santander De Quilichao,Fisico,17,8,35.647059,218770.588235,0.647059
4,1556,Zapatos,Antioquia-Medellin,Fisico,4,4,29.250000,147137.500000,1.000000


## Algoritmos de recomendacion

### Diseño de df para algoritmos

In [22]:
df_almacenes = pd.get_dummies(df_almacen, columns=['Categoria', 'Ubicacion_Almacen', 'Canal'])
df_almacenes.head()

,AlmacenCodigo,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,...,Ubicacion_Almacen_Valle Del Cauca-Roldanillo,Ubicacion_Almacen_Valle Del Cauca-Sevilla,Ubicacion_Almacen_Valle Del Cauca-Trujillo,Ubicacion_Almacen_Valle Del Cauca-Tulua,Ubicacion_Almacen_Valle Del Cauca-Union,Ubicacion_Almacen_Valle Del Cauca-Yumbo,Ubicacion_Almacen_Valle Del Cauca-Zarzal,Canal_Ecommerce,Canal_Fisico,Canal_Tienda Virtual
0,21083,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,16357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,24327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,16346,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1556,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [23]:
df_almacenes_index = df_almacenes.set_index('AlmacenCodigo')
df_almacenes_index.head()

,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,...,Ubicacion_Almacen_Valle Del Cauca-Roldanillo,Ubicacion_Almacen_Valle Del Cauca-Sevilla,Ubicacion_Almacen_Valle Del Cauca-Trujillo,Ubicacion_Almacen_Valle Del Cauca-Tulua,Ubicacion_Almacen_Valle Del Cauca-Union,Ubicacion_Almacen_Valle Del Cauca-Yumbo,Ubicacion_Almacen_Valle Del Cauca-Zarzal,Canal_Ecommerce,Canal_Fisico,Canal_Tienda Virtual
AlmacenCodigo,,,,,,,,,,,,,,,,,,,,,
21083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
24327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16346,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [24]:
df_almacenes_dummies = pd.get_dummies(df_almacen_detalle, columns=['Categoria', 'Ubicacion_Almacen', 'Canal'])
df_almacenes_dummies.head()

,AlmacenCodigo,TotalCreditos,cant_clientes,edad_clientes,Prom_Valorfactura,porcentaje_mujeres,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,...,Ubicacion_Almacen_Valle Del Cauca-Roldanillo,Ubicacion_Almacen_Valle Del Cauca-Sevilla,Ubicacion_Almacen_Valle Del Cauca-Trujillo,Ubicacion_Almacen_Valle Del Cauca-Tulua,Ubicacion_Almacen_Valle Del Cauca-Union,Ubicacion_Almacen_Valle Del Cauca-Yumbo,Ubicacion_Almacen_Valle Del Cauca-Zarzal,Canal_Ecommerce,Canal_Fisico,Canal_Tienda Virtual
0,21083,87,42,33.747126,103334.413793,0.965517,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,16357,47,39,34.319149,91275.340426,0.851064,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,24327,21,15,35.380952,140498.714286,0.904762,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,16346,17,8,35.647059,218770.588235,0.647059,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,1556,4,4,29.250000,147137.500000,1.000000,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [25]:
df_almacenes_stand = df_almacenes_dummies.copy()

# Estandarizar las variables numéricas utilizando la escala Min-Max
scaler = MinMaxScaler()
df_almacenes_stand[['TotalCreditos', 'cant_clientes', 'edad_clientes', 'Prom_Valorfactura', 'porcentaje_mujeres']] = scaler.fit_transform(df_almacenes_stand[['TotalCreditos', 'cant_clientes', 'edad_clientes', 'Prom_Valorfactura', 'porcentaje_mujeres']])
df_almacenes_stand.head()

,AlmacenCodigo,TotalCreditos,cant_clientes,edad_clientes,Prom_Valorfactura,porcentaje_mujeres,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,...,Ubicacion_Almacen_Valle Del Cauca-Roldanillo,Ubicacion_Almacen_Valle Del Cauca-Sevilla,Ubicacion_Almacen_Valle Del Cauca-Trujillo,Ubicacion_Almacen_Valle Del Cauca-Tulua,Ubicacion_Almacen_Valle Del Cauca-Union,Ubicacion_Almacen_Valle Del Cauca-Yumbo,Ubicacion_Almacen_Valle Del Cauca-Zarzal,Canal_Ecommerce,Canal_Fisico,Canal_Tienda Virtual
0,21083,0.115591,0.106771,0.260145,0.022831,0.861137,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,16357,0.061828,0.098958,0.271819,0.018112,0.759057,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,24327,0.026882,0.036458,0.293489,0.037377,0.806950,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,16346,0.021505,0.018229,0.298920,0.068012,0.577107,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,1556,0.004032,0.007812,0.168367,0.039976,0.891892,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
df_almacenes_stand_index = df_almacenes_stand.set_index('AlmacenCodigo')
df_almacenes_stand_index.head()

,TotalCreditos,cant_clientes,edad_clientes,Prom_Valorfactura,porcentaje_mujeres,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,...,Ubicacion_Almacen_Valle Del Cauca-Roldanillo,Ubicacion_Almacen_Valle Del Cauca-Sevilla,Ubicacion_Almacen_Valle Del Cauca-Trujillo,Ubicacion_Almacen_Valle Del Cauca-Tulua,Ubicacion_Almacen_Valle Del Cauca-Union,Ubicacion_Almacen_Valle Del Cauca-Yumbo,Ubicacion_Almacen_Valle Del Cauca-Zarzal,Canal_Ecommerce,Canal_Fisico,Canal_Tienda Virtual
AlmacenCodigo,,,,,,,,,,,,,,,,,,,,,
21083,0.115591,0.106771,0.260145,0.022831,0.861137,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16357,0.061828,0.098958,0.271819,0.018112,0.759057,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
24327,0.026882,0.036458,0.293489,0.037377,0.806950,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
16346,0.021505,0.018229,0.298920,0.068012,0.577107,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1556,0.004032,0.007812,0.168367,0.039976,0.891892,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Basado en contenido

### Similitud Coseno

#### Matriz de similitud entre los almacenes con Coseno

In [27]:
# Calcular la matriz de similitud entre los almacenes con Coseno
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(df_almacenes_index)

In [28]:
##Revision de los resultados de coseno
pd.DataFrame(cosine_sim).to_csv('Resultados/df_distancias.csv', index=False)

In [29]:
#
indices = pd.Series(df_almacenes.index, index=df_almacenes['AlmacenCodigo'])

In [30]:
indices[:10]
indices.shape

(1718,)

#### Funcion Lista de almacenes similares para un cliente usando **Coseno**

In [31]:
cosine_sim.shape

(1718, 1718)

In [32]:
import itertools

def listaAlmacenesSimilares_coseno(cliente, cosine_sim=cosine_sim):
    list_alm = df_credito.loc[df_credito['PersonaCodigo'] == cliente, 'AlmacenCodigo'].values
    idx = [indices[alm] for alm in list_alm]
    sim = []
    for i in idx:
        sim_scores = list(enumerate(cosine_sim[i]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = [score for score in sim_scores if score[0] != i]
        sim  = sim + sim_scores[1:4]
    alm_indices = [i[0] for i in sim]
    # Return the top 10 most similar movies
    return  df_almacenes['AlmacenCodigo'].iloc[alm_indices]

In [33]:
#Almacenes que compro 
df_credito.loc[df_credito['PersonaCodigo'] == 644573, 'AlmacenCodigo'].values

array([13394, 16197, 1937, 40877], dtype=object)

In [34]:
result_coseno = listaAlmacenesSimilares_coseno(644573) 
print(result_coseno)

190    16197
197    13396
208    23701
43     13394
197    13396
208    23701
26      1946
73     22043
119     1938
69     24881
82     28755
97     43984
Name: AlmacenCodigo, dtype: object


### Similitud Coseno Mas variables para el almacen

#### Matriz de similitud entre almacenes

In [35]:
cosine_sim_variables = cosine_similarity(df_almacenes_stand_index)
#
indices_alm = pd.Series(df_almacenes_stand.index, index=df_almacenes_stand['AlmacenCodigo'])
indices_alm[:10]
indices_alm.shape

(1718,)

#### Funcion lista de almacenes similares usando coseno 

In [36]:
def Almacenes_detalle_similares_coseno(cliente, cosine_sim_variables=cosine_sim_variables):
    list_alm = df_credito.loc[df_credito['PersonaCodigo'] == cliente, 'AlmacenCodigo'].values
    idx = [indices[alm] for alm in list_alm]
    sim = []
    for i in idx:
        sim_scores = list(enumerate(cosine_sim_variables[i]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = [score for score in sim_scores if score[0] != i]
        sim  = sim + sim_scores[1:4]
    alm_indices = [i[0] for i in sim[:6]]
    # Return the top 10 most similar movies
    return  df_almacenes['AlmacenCodigo'].iloc[alm_indices].tolist()

In [37]:
cli_s = 644573
result_coseno_var = Almacenes_detalle_similares_coseno(644573) 
print(result_coseno_var)

[16197, 16199, 1867, 13394, 16199, 13396]


In [38]:
df_alm_reco_coseno_var = pd.DataFrame()
for index in result_coseno_var:
    df_alm_reco_coseno_var = pd.concat([df_alm_reco_coseno_var, df_almacen[df_almacen['AlmacenCodigo']==int(index)]])

df_alm_reco_coseno_var

,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal
190,16197,Zapatos,Valle Del Cauca-Cali,Fisico
30,16199,Zapatos,Valle Del Cauca-Cali,Fisico
445,1867,Zapatos,Valle Del Cauca-Cali,Fisico
43,13394,Zapatos,Valle Del Cauca-Cali,Fisico
30,16199,Zapatos,Valle Del Cauca-Cali,Fisico
197,13396,Zapatos,Valle Del Cauca-Cali,Fisico


## Basado en Memoria

### Basado en memoria - Categoria Almacen

#### Construir Matriz Usuario-Item por categorias almacen

In [39]:
df_credito_categoria = pd.merge(df_credito,df_almacen)
df_credito_categoria = df_credito_categoria[['creditocodigo','AlmacenCodigo','PersonaCodigo','Categoria','Ubicacion_Almacen']]
df_credito_categoria.head()

,creditocodigo,AlmacenCodigo,PersonaCodigo,Categoria,Ubicacion_Almacen
0,21593606,13394,644573,Zapatos,Valle Del Cauca-Cali
1,21606717,13394,2953950,Zapatos,Valle Del Cauca-Cali
2,21641629,13394,1634570,Zapatos,Valle Del Cauca-Cali
3,21642687,13394,819835,Zapatos,Valle Del Cauca-Cali
4,21643177,13394,741905,Zapatos,Valle Del Cauca-Cali


In [40]:
df_categoria_d = df_credito_categoria[['PersonaCodigo','Categoria']]
df_categoria_d = pd.get_dummies(df_categoria_d, columns=['Categoria'])
df_categoria_d.head()

,PersonaCodigo,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,Categoria_Otro,Categoria_Salud y Belleza,Categoria_Servicios,Categoria_Tecnologia,Categoria_Vestuario,Categoria_Zapatos,Categoria_construccion
0,644573,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2953950,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1634570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,819835,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,741905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [41]:
df_persona_categoria_index = df_categoria_d.groupby('PersonaCodigo').agg('sum')
df_persona_categoria_index.head()

,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,Categoria_Otro,Categoria_Salud y Belleza,Categoria_Servicios,Categoria_Tecnologia,Categoria_Vestuario,Categoria_Zapatos,Categoria_construccion
PersonaCodigo,,,,,,,,,,,,,,,,,
587,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0
3384,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,0
7170,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
7860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0
12519,0,0,0,5,0,0,0,0,0,0,0,2,0,0,0,1,0


In [42]:
print('Numero de filas: ',df_persona_categoria_index.shape[0])
print('Numero de clientes: ',df_persona_categoria_index.index.nunique())

Numero de filas:  9878
Numero de clientes:  9878


#### Matriz de similitud con la correlación de Pearson

In [43]:
# Calcular la similitud entre los clientes usando la correlación de Pearson
similitud = np.corrcoef(df_persona_categoria_index)

In [44]:
df_persona_categoria = df_persona_categoria_index.reset_index()
df_persona_categoria.head()

,PersonaCodigo,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,Categoria_Otro,Categoria_Salud y Belleza,Categoria_Servicios,Categoria_Tecnologia,Categoria_Vestuario,Categoria_Zapatos,Categoria_construccion
0,587,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0
1,3384,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5,0
2,7170,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,7860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2,0
4,12519,0,0,0,5,0,0,0,0,0,0,0,2,0,0,0,1,0


In [45]:
indices_persona = pd.Series(df_persona_categoria.index, index=df_persona_categoria['PersonaCodigo'])

In [46]:
# Crear el diccionario con el índice de la columna y su nombre
column_dict = {index: column for index, column in enumerate(df_persona_categoria.columns.str.replace('Categoria_', ''))}
print(column_dict)

{0: 'PersonaCodigo', 1: 'Accesorios', 2: 'Bebes', 3: 'Bolsos', 4: 'Bolsos y Vestuario', 5: 'Comidas y Bares', 6: 'Construccion', 7: 'Deportes', 8: 'Hogar', 9: 'Hogar y Tecnologia', 10: 'Movilidad', 11: 'Otro', 12: 'Salud y Belleza', 13: 'Servicios', 14: 'Tecnologia', 15: 'Vestuario', 16: 'Zapatos', 17: 'construccion'}


#### Algoritmo de recomendacion basado en memoria categoria 

In [47]:
# Función para obtener las recomendaciones para un cliente dado
def obtener_recomendaciones(codPersona):
    cliente = indices_persona[codPersona]
    # Obtener las similitudes del cliente dado con todos los demás clientes
    similitudes_cliente = similitud[cliente]
    #Calcular el promedio ponderado de las compras de los clientes similares
    recomendaciones = np.dot(similitudes_cliente, df_persona_categoria) / np.sum(np.abs(similitudes_cliente))
    #Excluir las categorías en las que el cliente ya ha compra
    categ_compradas = df_persona_categoria.iloc[cliente].to_numpy().astype(bool)
    recomendaciones[categ_compradas] = 0
    #Ordenar las recomendaciones de mayor a menor
    indices_recomendaciones = np.argsort(recomendaciones)[::-1]
    sorted_categories = [column_dict[index] for index in indices_recomendaciones[0:2]] 
    
    #Ubicacion Cliente segun compras
    list_ub = df_credito_categoria.loc[df_credito_categoria['PersonaCodigo'] == codPersona, 'Ubicacion_Almacen'].unique()
    df_alm_recomendados = pd.DataFrame()
    for ub in list_ub:
        df_alm_cliente = df_almacen[df_almacen['Ubicacion_Almacen']==ub]
        for cat in sorted_categories:
            df_alm_recom = df_alm_cliente[df_alm_cliente['Categoria']==cat].head(2)[['AlmacenCodigo','Categoria','Ubicacion_Almacen']]
            df_alm_recomendados = pd.concat([df_alm_recomendados, df_alm_recom])

    return df_alm_recomendados

In [48]:
# Ejemplo de uso: obtener las recomendaciones para el cliente 
cli= 644573
print("Recomendaciones para el cliente:", obtener_recomendaciones(cli))
df_persona_categoria[df_persona_categoria['PersonaCodigo']==cli]

Recomendaciones para el cliente:      AlmacenCodigo  Categoria           Ubicacion_Almacen
0            21083  Vestuario        Valle Del Cauca-Cali
110          22511  Vestuario        Valle Del Cauca-Cali
124          20422      Hogar        Valle Del Cauca-Cali
1351         22361      Hogar        Valle Del Cauca-Cali
1063         18641  Vestuario  Valle Del Cauca-Roldanillo


,PersonaCodigo,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,Categoria_Otro,Categoria_Salud y Belleza,Categoria_Servicios,Categoria_Tecnologia,Categoria_Vestuario,Categoria_Zapatos,Categoria_construccion
861,644573,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,2,0


In [49]:
df_almacen.head()

,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal
0,21083,Vestuario,Valle Del Cauca-Cali,Fisico
1,16357,Zapatos,Caqueta-Florencia,Fisico
2,24327,Vestuario,Antioquia-Sabaneta,Fisico
3,16346,Bolsos y Vestuario,Cauca-Santander De Quilichao,Fisico
4,1556,Zapatos,Antioquia-Medellin,Fisico


In [50]:
# Ejemplo de uso: obtener las recomendaciones para el cliente 
cli = 741905
print("Recomendaciones para el cliente:", obtener_recomendaciones(cli))
df_persona_categoria[df_persona_categoria['PersonaCodigo']==cli]

Recomendaciones para el cliente:     AlmacenCodigo           Categoria     Ubicacion_Almacen
0           21083           Vestuario  Valle Del Cauca-Cali
110         22511           Vestuario  Valle Del Cauca-Cali
23           1945  Bolsos y Vestuario  Valle Del Cauca-Cali
26           1946  Bolsos y Vestuario  Valle Del Cauca-Cali


,PersonaCodigo,Categoria_Accesorios,Categoria_Bebes,Categoria_Bolsos,Categoria_Bolsos y Vestuario,Categoria_Comidas y Bares,Categoria_Construccion,Categoria_Deportes,Categoria_Hogar,Categoria_Hogar y Tecnologia,Categoria_Movilidad,Categoria_Otro,Categoria_Salud y Belleza,Categoria_Servicios,Categoria_Tecnologia,Categoria_Vestuario,Categoria_Zapatos,Categoria_construccion
1057,741905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0


### Basado en memoria - Almacen

#### Construir Matriz Usuario-Item por almacen

In [51]:
df_credito_d = df_credito[['PersonaCodigo','AlmacenCodigo']]
df_credito_d = pd.get_dummies(df_credito_d, columns=['AlmacenCodigo'])
df_persona_almacen_index = df_credito_d.groupby('PersonaCodigo').agg('sum')
df_persona_almacen_index.head()

/var/folders/vn/gyqsnwt95575hv9b5y4_sg8c0000gn/T/ipykernel_610/35309178.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_credito_d = pd.get_dummies(df_credito_d, columns=['AlmacenCodigo'])


,AlmacenCodigo_10,AlmacenCodigo_17,AlmacenCodigo_35,AlmacenCodigo_194,AlmacenCodigo_303,AlmacenCodigo_318,AlmacenCodigo_319,AlmacenCodigo_322,AlmacenCodigo_323,AlmacenCodigo_331,...,AlmacenCodigo_44121,AlmacenCodigo_44138,AlmacenCodigo_44142,AlmacenCodigo_44158,AlmacenCodigo_44173,AlmacenCodigo_44482,AlmacenCodigo_44620,AlmacenCodigo_44717,AlmacenCodigo_44777,AlmacenCodigo_45159
PersonaCodigo,,,,,,,,,,,,,,,,,,,,,
587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Matriz de similitud con la correlación de Pearson Almacen

In [52]:
# Calcular la similitud entre los clientes usando la correlación de Pearson
similitud_almacen = np.corrcoef(df_persona_almacen_index)

In [53]:
df_persona_almacen = df_persona_almacen_index.reset_index()
df_persona_almacen.head()

,PersonaCodigo,AlmacenCodigo_10,AlmacenCodigo_17,AlmacenCodigo_35,AlmacenCodigo_194,AlmacenCodigo_303,AlmacenCodigo_318,AlmacenCodigo_319,AlmacenCodigo_322,AlmacenCodigo_323,...,AlmacenCodigo_44121,AlmacenCodigo_44138,AlmacenCodigo_44142,AlmacenCodigo_44158,AlmacenCodigo_44173,AlmacenCodigo_44482,AlmacenCodigo_44620,AlmacenCodigo_44717,AlmacenCodigo_44777,AlmacenCodigo_45159
0,587,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3384,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7170,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12519,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
indices_persona_alm = pd.Series(df_persona_almacen.index, index=df_persona_almacen['PersonaCodigo'])

In [55]:
column_dict_alm = {index: column for index, column in enumerate(df_persona_almacen.columns.str.replace('AlmacenCodigo_', ''))}
print(column_dict_alm)

{0: 'PersonaCodigo', 1: '10', 2: '17', 3: '35', 4: '194', 5: '303', 6: '318', 7: '319', 8: '322', 9: '323', 10: '331', 11: '332', 12: '333', 13: '334', 14: '335', 15: '469', 16: '484', 17: '623', 18: '656', 19: '660', 20: '753', 21: '769', 22: '882', 23: '883', 24: '884', 25: '888', 26: '889', 27: '890', 28: '891', 29: '892', 30: '893', 31: '894', 32: '896', 33: '910', 34: '911', 35: '916', 36: '927', 37: '978', 38: '979', 39: '1008', 40: '1090', 41: '1280', 42: '1284', 43: '1285', 44: '1324', 45: '1326', 46: '1424', 47: '1433', 48: '1438', 49: '1445', 50: '1450', 51: '1458', 52: '1459', 53: '1460', 54: '1461', 55: '1463', 56: '1464', 57: '1465', 58: '1466', 59: '1467', 60: '1469', 61: '1470', 62: '1479', 63: '1480', 64: '1488', 65: '1528', 66: '1535', 67: '1537', 68: '1538', 69: '1539', 70: '1540', 71: '1541', 72: '1542', 73: '1544', 74: '1545', 75: '1548', 76: '1549', 77: '1552', 78: '1555', 79: '1556', 80: '1563', 81: '1574', 82: '1575', 83: '1595', 84: '1597', 85: '1598', 86: '1599

#### Algoritmo de recomendacion basado en memoria almacen

In [56]:
# Función para obtener las recomendaciones para un cliente dado
def obtener_recomendaciones_almacenes(codPersona):
    cliente = indices_persona_alm[codPersona]
    # Obtener las similitudes del cliente dado con todos los demás clientes
    similitudes_cliente = similitud_almacen[cliente]
    #Calcular el promedio ponderado de las compras de los clientes similares
    recomendaciones = np.dot(similitudes_cliente, df_persona_almacen) / np.sum(np.abs(similitudes_cliente))
    #Excluir las categorías en las que el cliente ya ha compra
    #recomendaciones[cliente] = 0
    alm_compradas = df_persona_almacen.iloc[cliente].to_numpy().astype(bool)
    recomendaciones[np.where(alm_compradas)] = 0
    #Ordenar las recomendaciones de mayor a menor
    indices_recomendaciones = np.argsort(recomendaciones)[::-1]
    sorted_alm = [column_dict_alm[index] for index in indices_recomendaciones[0:5]] 
    #sorted_alm = [index for index in indices_recomendaciones[0:5]]
    df_alm_reco = pd.DataFrame()
    for index in sorted_alm:
        df_alm_reco = pd.concat([df_alm_reco, df_almacen[df_almacen['AlmacenCodigo']==int(index)]])


    

    return df_alm_reco

In [57]:
cli= 644573
df_credito_categoria[df_credito_categoria['PersonaCodigo']==cli]

,creditocodigo,AlmacenCodigo,PersonaCodigo,Categoria,Ubicacion_Almacen
0,21593606,13394,644573,Zapatos,Valle Del Cauca-Cali
1571,21593103,16197,644573,Zapatos,Valle Del Cauca-Cali
9505,21592240,1937,644573,Bolsos y Vestuario,Valle Del Cauca-Cali
52446,21815693,40877,644573,Movilidad,Valle Del Cauca-Roldanillo


In [58]:
# Ejemplo de uso: obtener las recomendaciones para el cliente 
print("Recomendaciones para el cliente:")
obtener_recomendaciones_almacenes(cli)


Recomendaciones para el cliente:


,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal
110,22511,Vestuario,Valle Del Cauca-Cali,Fisico
672,17911,Vestuario,Valle Del Cauca-Cali,Fisico
1155,15726,Vestuario,Valle Del Cauca-Cali,Fisico
310,1942,Bolsos y Vestuario,Valle Del Cauca-Cali,Fisico
841,2538,Vestuario,Valle Del Cauca-Cali,Fisico


## Modelado con libreria Suprise

In [59]:
# Import surprise modules
from surprise import Dataset
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import SVD, SVDpp, NMF, SlopeOne, CoClustering,KNNBasic, KNNBaseline, KNNWithZScore, KNNWithMeans,BaselineOnly, NormalPredictor
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [60]:
df_persona_almacen_bin = df_persona_almacen_index.clip(upper=1)

In [61]:
df_persona_almacen_bin.head()

,AlmacenCodigo_10,AlmacenCodigo_17,AlmacenCodigo_35,AlmacenCodigo_194,AlmacenCodigo_303,AlmacenCodigo_318,AlmacenCodigo_319,AlmacenCodigo_322,AlmacenCodigo_323,AlmacenCodigo_331,...,AlmacenCodigo_44121,AlmacenCodigo_44138,AlmacenCodigo_44142,AlmacenCodigo_44158,AlmacenCodigo_44173,AlmacenCodigo_44482,AlmacenCodigo_44620,AlmacenCodigo_44717,AlmacenCodigo_44777,AlmacenCodigo_45159
PersonaCodigo,,,,,,,,,,,,,,,,,,,,,
587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3384,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7170,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12519,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df_mui = df_persona_almacen_bin.reset_index().melt(id_vars=['PersonaCodigo'], var_name='Almacen', value_name='Compro')

In [63]:
df_mui['Almacen'] =df_mui['Almacen'].replace('AlmacenCodigo_', '', regex=True)

In [84]:
df_mui.head()

,PersonaCodigo,Almacen,Compro
0,587,10,0
1,3384,10,0
2,7170,10,0
3,7860,10,0
4,12519,10,0


In [56]:
cliente_id = 644573

#### Reader, data, trainset y testset

In [65]:
# Crear el lector
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_mui, reader)


In [58]:
trainset, testset = train_test_split(data, test_size=0.2)


In [244]:
#elementos = data.build_full_trainset().all_items()

In [246]:
# Crear el conjunto de prueba para el cliente dado
#testset_cliente = [(cliente_id, elemento, 0) for elemento in elementos]

#### Modelo KNNBasic

In [232]:
model_knn = KNNBasic()

In [234]:
model_knn.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [247]:
prediccion_knn = model_knn.test(testset_cliente)

In [ ]:


predicciones_cliente_knn = [pred for pred in prediccion_knn if pred[0] == cliente_id ]

predicciones_cliente_knn.sort(key=lambda x: x.est, reverse=True)
almacenes_recomendados_knn = [pred[1] for pred in predicciones_cliente_knn]
print("Almacenes recomendados para el cliente:", almacenes_recomendados_knn)

In [ ]:
df_recommended_stores_knn = pd.DataFrame()
for index in almacenes_recomendados_knn:
    df_recommended_stores_knn = pd.concat([df_recommended_stores_knn, df_almacen[df_almacen['AlmacenCodigo']==index]])

#### Modelo SVD

In [59]:
# Entrenar el modelo SVD
model_svd = SVD()
model_svd.fit(trainset)


In [60]:
# Obtener las predicciones para el conjunto de prueba
predictions_svd = model_svd.test(testset)

In [82]:
rmse = accuracy.rmse(predictions_svd)

print("RMSE: ", rmse)

RMSE: 0.0457
RMSE:  0.0457315894404434


In [76]:
# Función para obtener las recomendaciones para un cliente dado
def obtener_recomendaciones_almacenes_svd(codPersona):
    almacenes_no_comprados = df_mui[(df_mui['PersonaCodigo'] != codPersona) & (df_mui['Compro'] != 1)]['Almacen'].unique()
    almacenes_comprados = df_mui[(df_mui['PersonaCodigo'] == codPersona) & (df_mui['Compro'] == 1)]['Almacen'].unique()
    almacenes_no_comprados = np.setdiff1d(almacenes_no_comprados, almacenes_comprados)

    # Obtener las predicciones para los almacenes no comprados por el cliente
    predicciones = [model_svd.predict(codPersona, almacen) for almacen in almacenes_no_comprados]

    # Ordenar las predicciones de mayor a menor
    predicciones.sort(key=lambda x: x.est, reverse=True)

    # Obtener los nombres de los almacenes recomendados para el cliente
    almacenes_recomendados = [prediccion.iid for prediccion in predicciones]

    return almacenes_recomendados[:5]

In [78]:

# Obtener las recomendaciones para el cliente dado
top_recommendations_svd = obtener_recomendaciones_almacenes_svd(cliente_id)

# Imprimir las recomendaciones
print("Almacenes recomendados para el cliente", cliente_id, ":")
print(top_recommendations_svd)

Almacenes recomendados para el cliente 644573 :
['31715', '37495', '41128', '14138', '20787']


In [80]:
df_recommended_stores_svd = pd.DataFrame()
for index in top_recommendations_svd:
    df_recommended_stores_svd = pd.concat([df_recommended_stores_svd, df_almacen[df_almacen['AlmacenCodigo']==int(index)]])

df_recommended_stores_svd

,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal
16,31715,Hogar y Tecnologia,Antioquia-Envigado,Fisico
353,37495,Bolsos y Vestuario,Bogotá D.c.-Bogotá,Ecommerce
717,41128,Servicios,Antioquia-Envigado,Fisico
67,14138,Vestuario,Huila-Neiva,Fisico
718,20787,Vestuario,Valle Del Cauca-Tulua,Fisico


##### Hiperparametros SVD

In [66]:
from surprise.model_selection import GridSearchCV

# Definir los parámetros a probar
param_grid = {'n_factors': [50, 100, 200],
              'learning_rate': [0.001, 0.005, 0.01]}

# Crear el objeto GridSearchCV
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

# Entrenar el modelo con diferentes combinaciones de hiperparámetros
grid_search.fit(data)

# Obtener los mejores parámetros y el mejor resultado
best_params = grid_search.best_params['rmse']
best_score = grid_search.best_score['rmse']

print("Mejores parámetros: ", best_params)
print("Mejor puntuación: ", best_score)

TypeError: __init__() got an unexpected keyword argument 'learning_rate'

##### Personalizacion del modelo

In [ ]:

# Crear el modelo SVD personalizado
model_svd_pr = SVD(n_factors=100, reg_all=0.1, random_state=42)

# Entrenar el modelo con los datos
model_svd_pr.fit(trainset)

# Obtener las predicciones para el conjunto de prueba
predictions_svd_pr = model_svd_pr.test(testset)

# Evaluar las predicciones utilizando una métrica, por ejemplo, RMSE
from surprise import accuracy
rmse = accuracy.rmse(predictions_svd_pr)

print("RMSE: ", rmse)

#### Modelo SVD++

In [250]:
# Crear y entrenar el modelo SVD++
model_svdpp = SVDpp()
model_svdpp.fit(trainset)



: 

: 

In [ ]:
prediccion_svdpp = model_svdpp.test(testset_cliente)

In [ ]:
cliente_id = 644573

In [ ]:

predicciones_cliente_svdpp = [pred for pred in prediccion_svdpp if pred[0] == cliente_id ]

predicciones_cliente_svdpp.sort(key=lambda x: x.est, reverse=True)
almacenes_recomendados_svdpp = [pred[1] for pred in predicciones_cliente_svdpp[:5]]
almacenes_recomendados_svdpp

In [ ]:
print("Predicción para el cliente", cliente_id, "usando SVD++:")
for pred in prediccion_svdpp:
    print(pred)

#### CoClustering

In [ ]:
# Crear y entrenar el modelo Co-Clustering
model_coclustering = CoClustering()
model_coclustering.fit(trainset)

# Realizar predicciones en el conjunto de prueba
predictions = model_coclustering.test(testset)

In [ ]:
prediccion_coclustering = model_coclustering.test(testset_cliente)

In [ ]:
print("Predicción para el cliente", cliente_id, "usando Co-Clustering:")
for pred in prediccion_coclustering:
    print(pred)


#### NMF

In [ ]:
# Crear y entrenar el modelo NMF
model_nmf = NMF()
model_nmf.fit(trainset)

# Realizar predicciones en el conjunto de prueba


In [ ]:
prediccion_nmf = model_nmf.test(testset_cliente)

In [ ]:
print("Predicción para el cliente", cliente_id, "usando NMF:")
for pred in prediccion_nmf:
    print(pred)

In [ ]:
# Evaluar el modelo utilizando validación cruzada
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

## Modelado con otra

In [93]:
from scipy.sparse.linalg import svds

# Ejemplo de datos
data = {
    'Persona': ['A', 'B', 'C', 'D', 'E'],
    'Categoria1': [1, 1, 0, 1, 0],
    'Categoria2': [0, 1, 1, 0, 1],
    'Categoria3': [1, 0, 1, 0, 0]
}

df = pd.DataFrame(data)
df.set_index('Persona', inplace=True)

# Convertir los datos a una matriz sparse
sparse_data = df_persona_almacen_bin.values.astype(float)
sparse_data = sparse_data.T

# Descomposición en valores singulares (SVD)
U, sigma, Vt = svds(sparse_data, k=2)  # k es el número de dimensiones latentes

# Reconstruir la matriz original
sigma = np.diag(sigma)
predicted_data = np.dot(np.dot(U, sigma), Vt)

# Crear un DataFrame con las predicciones
predicted_df = pd.DataFrame(predicted_data.T, columns=df_persona_almacen_bin.columns, index=df_persona_almacen_bin.index)



               AlmacenCodigo_10  AlmacenCodigo_17  AlmacenCodigo_35  \
PersonaCodigo                                                         
587                1.003462e-07      5.247913e-07      2.008450e-05   
3384               6.171002e-10      3.934116e-09      1.260227e-07   
7170               5.348819e-11      4.181098e-10      1.119695e-08   
7860               2.623297e-10      1.229447e-08      9.127612e-08   
12519              2.985249e-08      2.387266e-07      6.268244e-06   
...                         ...               ...               ...   
4986815            1.092075e-09      7.794473e-09      2.259752e-07   
4987291            2.973110e-08      2.883007e-07      6.422169e-06   
4997924            4.985008e-11      3.906191e-10      1.043873e-08   
5001776            4.992036e-10      3.036610e-09      1.014283e-07   
5039882            5.758725e-10      3.919922e-09      1.184858e-07   

               AlmacenCodigo_194  AlmacenCodigo_303  AlmacenCodigo_318  \
Pe

In [94]:
def recommend_categories(user, predicted_df, top_n=3):
    user_predictions = predicted_df.loc[user]
    recommended_categories = user_predictions.sort_values(ascending=False).head(top_n).index.tolist()

    return recommended_categories

# Ejemplo de recomendación para el usuario 'A'
user = 644573
recommendations = recommend_categories(user, predicted_df)
print(f"Recomendaciones para el usuario {user}: {recommendations}")

Recomendaciones para el usuario 644573: ['AlmacenCodigo_20787', 'AlmacenCodigo_15124', 'AlmacenCodigo_21961']


In [96]:
df_recommended_svd = pd.DataFrame()
list=[20787,15124,21961]
for index in list:
    df_recommended_svd = pd.concat([df_recommended_svd, df_almacen[df_almacen['AlmacenCodigo']==int(index)]])

df_recommended_svd

,AlmacenCodigo,Categoria,Ubicacion_Almacen,Canal
718,20787,Vestuario,Valle Del Cauca-Tulua,Fisico
701,15124,Vestuario,Valle Del Cauca-Tulua,Fisico
146,21961,Vestuario,Valle Del Cauca-Tulua,Fisico
